In [1]:
import json
import pandas as pd
import os
import re
import string
from categories import CATEGORIES
from transformers import BertForSequenceClassification
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import AutoTokenizer

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

from transformers import TrainingArguments, Trainer
from datasets import Dataset

import numpy as np

DATA_PATH = "arxiv-metadata-oai-snapshot.json"
YEAR_CUTOFF = 2012
YEAR_PATTERN = r"(19|20[0-9]{2})"
ML_CATEGORY = list(CATEGORIES.keys())

In [2]:
def process(paper: dict):
    paper = json.loads(paper)
    if paper['journal-ref']:
        years = [int(year) for year in re.findall(YEAR_PATTERN, paper['journal-ref'])]
        years = [year for year in years if (year <= 2022 and year >= 1991)]
        year = min(years) if years else None
    else:
        year = None
    return {
        'id': paper['id'],
        'title': paper['title'],
        'year': year,
        'authors': paper['authors'],
        'categories': ','.join(paper['categories'].split(' ')),
        'abstract': paper['abstract']
    }

def papers():
    with open(DATA_PATH, 'r') as f:
        for paper in f:
            paper = process(paper)
            if paper['year']:
                if paper['year'] >= YEAR_CUTOFF:
                    yield paper

In [3]:
df = pd.DataFrame(papers())
len(df)

408773

In [5]:
df.head(3)

,id,title,year,authors,categories,abstract,text
0,0704.0304,The World as Evolving Information,2012,Carlos Gershenson,"[cs.IT, cs.AI, math.IT, q-bio.PE]",This paper discusses the benefits of describ...,The World as Evolving Information This paper...
1,0704.2744,Nahm transform and parabolic minimal Laplace t...,2012,Szilard Szabo,[math.AG],We prove that Nahm transform for integrable ...,Nahm transform and parabolic minimal Laplace t...
2,0704.2768,Heat Equations and the Weighted $\bar\partial$...,2012,Andrew Raich,"[math.AP, math.CV]",The purpose of this article is to establish ...,Heat Equations and the Weighted $\bar\partial$...


In [7]:
df['text'] = df['title'] + ' ' + df['abstract']
# df['categories'] = df['categories'].apply(lambda x: x.split(','))

In [10]:
df.iloc[0].categories

['cs.IT', 'cs.AI', 'math.IT', 'q-bio.PE']

In [7]:
# Split into train and test

from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2)

In [8]:
def get_tokenizer(tokenizer_model):
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    tokenizer = AutoTokenizer.from_pretrained(tokenizer_model)
    return tokenize_function, tokenizer

tokenize_function, tokenizer = get_tokenizer('bert-base-uncased')

In [9]:
mlb = MultiLabelBinarizer()
mlb.fit(df_train['categories'])

MultiLabelBinarizer()

In [10]:
def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]

  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)

  encoded_categories = mlb.transform(examples['categories']).astype(float)

  encoding["labels"] = encoded_categories

  return encoding

In [11]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels=len(mlb.classes_), 
    problem_type="multi_label_classification"
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [39]:
df_train_hf = Dataset.from_pandas(df_train[['text', 'categories']].head(100))
tokenized_train = df_train_hf.map(preprocess_data, batched=True)

df_test_hf = Dataset.from_pandas(df_test[['text', 'categories']].head(100))
tokenized_test = df_test_hf.map(preprocess_data, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [40]:
# Debugging - get inverse transform

print("Reversed", mlb.inverse_transform(np.asarray(tokenized_test[0]['labels']).reshape(1, -1)))
print("Original categories", tokenized_test[0]['categories'])

Reversed [('astro-ph.IM', 'astro-ph.SR')]
Original categories ['astro-ph.SR', 'astro-ph.IM']


In [41]:
# Adaptation: https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb 
# 
batch_size = 16
metric_name = "f1"

args = TrainingArguments(
    f"paper-multilabel-finetuning",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)


# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [42]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [43]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text, categories. If __index_level_0__, text, categories are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/paulo/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 100
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 48
  Gradient Accumulation steps = 1
  Total optimization steps = 15
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/home/paulo/.local/li

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.051140,0.000000,0.500000,0.000000
2,No log,0.051082,0.000000,0.500000,0.000000
3,No log,0.051024,0.000000,0.500000,0.000000
4,No log,0.050978,0.000000,0.500000,0.000000
5,No log,0.050960,0.000000,0.500000,0.000000


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text, categories. If __index_level_0__, text, categories are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 48
Saving model checkpoint to paper-multilabel-finetuning/checkpoint-3
Configuration saved in paper-multilabel-finetuning/checkpoint-3/config.json
Model weights saved in paper-multilabel-finetuning/checkpoint-3/pytorch_model.bin
tokenizer config file saved in paper-multilabel-finetuning/checkpoint-3/tokenizer_config.json
Special tokens file saved in paper-multilabel-finetuning/checkpoint-3/special_tokens_map.json
/home/paulo/.local/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze an

TrainOutput(global_step=15, training_loss=0.05476108392079671, metrics={'train_runtime': 39.6836, 'train_samples_per_second': 12.6, 'train_steps_per_second': 0.378, 'total_flos': 32935833984000.0, 'train_loss': 0.05476108392079671, 'epoch': 5.0})

In [25]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text, categories. If __index_level_0__, text, categories are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 48
/home/paulo/.local/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.09641064703464508,
 'eval_f1': 0.0,
 'eval_roc_auc': 0.5,
 'eval_accuracy': 0.0,
 'eval_runtime': 6.8075,
 'eval_samples_per_second': 146.896,
 'eval_steps_per_second': 3.085,
 'epoch': 15.0}

In [26]:
text = df['text'].iloc[5]

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k, v in encoding.items()}

outputs = trainer.model(**encoding)
logits = outputs.logits

In [31]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.1)] = 1

In [32]:
print(text)
print(mlb.inverse_transform(predictions.reshape(1, -1)))

Suppression of growth by multiplicative white noise in a parametric
  resonant system   The author studied the growth of the amplitude in a Mathieu-like equation
with multiplicative white noise. The approximate value of the exponent at the
extremum on parametric resonance regions was obtained theoretically by
introducing the width of time interval, and the exponents were calculated
numerically by solving the stochastic differential equations by a symplectic
numerical method. The Mathieu-like equation contains a parameter $\alpha$ that
is determined by the intensity of noise and the strength of the coupling
between the variable and the noise. The value of $\alpha$ was restricted not to
be negative without loss of generality. It was shown that the exponent
decreases with $\alpha$, reaches a minimum and increases after that. It was
also found that the exponent as a function of $\alpha$ has only one minimum at
$\alpha \neq 0$ on parametric resonance regions of $\alpha = 0$. This minimum
va

In [33]:
import pickle


with open('mlb.pickle', 'wb') as handle:
    pickle.dump(mlb, handle, protocol=pickle.HIGHEST_PROTOCOL)

